<a href="https://colab.research.google.com/github/Benned-H/Summer2019/blob/master/Speech%20and%20Language%20Processing/Chapter_3_Words_%26_Transducers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Words & Transducers (p. 51)

We've seen how to create plural *woodchucks*, but plural *foxes*, *peccaries*, and *geese* are all irregular forms. *Fish* even retains its spelling in the plural! To search for these forms, we need **orthographic rules** (*-y* endings turn to *-ie* in the plural) and **morphological rules** (*fish* has a null plural, *geese* has a changed vowel).

**Morphological parsing**, or stemming, is the problem of recognizing and separating morphemes of words in order to build structured representations. **Parsing** itself entails taking an input and producing some linguistic structure for it. In this case, we could take the **surface form** *going* and parse it into ```VERB-go + GERUND-ing```.

This technique is especially important for morphologically complex languages, as we want to search for a single word and find its many inflections as well. It's also used in POS-tagging and machine translation. To solve morphological parsing, couldn't we use a dictionary of valid forms? The issue is that many affixes are **productive**, e.g. we can take a word *Google* and create *Googled* or *fax* and create *faxing*. Both *-ing* and *-s* can attach to almost any verb or noun, respectively. Other languages make the dictionary idea even more infeasible.

*Overview*: In this chapter we'll survey the morphology of English and some other languages and then introduce **finite-state transducers**, a crucial technology in many areas of this text. We'll also introduce related algorithms like **stemming**, which maps from derived words to their stem. Similarly, the problem of mapping *sing*, *sang*, and *sung* to the same **lemma** is called **lemmatization**. Next, **tokenization** is the task of separating words from a text. Whitespace isn't infallible, as words like *New York* or *rock 'n' roll* contain spaces, yet two-word contractions often *don't*. Finally, we sometimes want to see how similar two words are orthographically (spelling-wise), and the **minimum edit distance** algorithm works for such spell-checking applications.

## 3.1 Survey of (Mostly) English Morphology (p. 53)

Morphology studies how words are built from smaller units of meaning called **morphemes**. These "minimal meaning-breaing units" fall into two broad classes, **stems** (or roots, bases) and **affixes**. Intuitively, stems are the main morphemes of words while affixes add additional kinds of meaning. We can further divide affixes into **prefixes**, **suffixes**, **infixes**, and **circumfixes**. Prefixes precede their stem, suffixes follow, circumfixes do both, and infixes are inserted into the stem. The first two might be obvious in English, but we need to look to German for a circumfix: the past participle of some verbs is formed by adding *ge*- to the beginning and -*t* to the end of the base, e.g. *sagen* (to say) becomes *gesagt* (said).

We can stack a few affixes in English, maybe *un-* *believe* *-able* *-ly* creating *unbelievably*, but other languages like Turkish can have nine or ten affixes. Such many-affix langauges are called **agglutinative**. We can also find define a few ways, such as **inflection**, **derivation**, **compounding**, and **cliticization**, to create new words by combining morphemes. **Inflection** combines a stem with a grammatical morpheme, usually in the same word class as the original word. **Derivation** combines a stem with a grammatical morpheme resulting in a different class of word, like *create* -*tion* becoming *creation*. **Compounding** combines multiple stems, and **cliticization** combine a word stem with a **clitic**. Clitics are morphemes that act like words but are reduced in form, like *'ve'* in *I've* or the French definite article *l'* in *l'opera*.

### Inflectional Morphology

English has a limited inflectional system. English nouns have two kinds of inflection: **plural** and **possessive**. Many English nouns can either appear in a bare **singular** form or take a plural suffix. The plural suffix often takes the form -*s*, but can be -*es* or change -*y* to -*ies* depending on the end of words. The possessive is realized by an apostrophe + -*s* in many cases.


Verbal inflection brings a few more options to English, as English has three kinds of verbs: **main verbs** (*eat*, *sleep*, *impeach*), **modal verbs** (*can*, *will*, *should*), and **primary verbs** (*be*, *have*, *do*) (terms of Quirk et al., 1985). Most verbs are **regular**, meaning they have the same endings marking the same functions. These four forms are the stem (*walk*, *try*), -*s* form (*walks*, *tries*), -*ing* participle (*walking*, *trying*), and past form (*walked*, *tried*). This class of verbs is **productive**, meaning it automatically includes new words introduced to the language.

**Irregular verbs** have different, abnormal forms of inflection. These can have as many as eight forms, few as three, but are often some of the most often-used words. As an example, *eat* has the past *ate* as well as the -*ed*/-*en* participle *eaten*. The usage of all of these forms is as follows:
* -*s* is used to distinguish 3rd-person singular.
* Stem forms are used as infinitives.
* -*ing* is used in the **progressive** to mark ongoing activity, or when the verb is used as a noun as a **gerund** (*Fishing* is fun).
* -*ed*/-*en* is used in the **perfect** or passive (He's *eaten* already; The apple was *eaten*).

We also need to consider spelling differences as we attach these different suffixes. Single consonants are doubled, a final 'c' becomes 'ck', delete a silent final -*e*, etc.

### Derivational Morphology

As if to make up for its simple inflectional morphology, English has quite a complex derivational morphology. Derivation combines a stem with some morpheme, and the result is typically of a *different* class. **Nominalization** is the formation of new nouns, for example adding -*ation* to verbs ending in -*ize*, -*er* to a verb, or -*ness* to an adjective. Adjectives can also be derived, e.g. -*able* from a verb or -*less* from a noun.

Derivation is more complex for a few reasons, including that it's less productive: suffixes only attach to certain words, and the changes in meaning are often inconsistent and subtle.

### Cliticization

Recall: clitics are those units between an affix and a word (they seem like contractions to me). They phonologically act like affixes, but syntactically like words. Clitics before a word are **proclitics** and those after are **enclitics**. English makes them a bit easier to parse thanks to the apostrophe, but other languages (e.g. Arabic) could have multiple single-letter proclitics *and* enclitics.

### Non-concatenative Morphology

The morphology we've discussed so far, strings of morphemes concatenated together, is called **concatenative morphology**. A more complex mophology might be **non-concatenative**. Tagalog infixation is one example, where two morphemes intermingle in a non-sequential way. Another kind is **templatic morphology** (or **root-and-pattern**), which is common in Arabic, Hebrew, and other Semitic languages.

**Eg**: Take Hebrew. A verb is constructed from two components: a root, which is three consonants (CCC), and a template, which gives the ordering of the consonants and vowels. The Hebrew tri-consonantal root *lmd*, meaning 'learn' or 'study' can be combined in the following ways:
* Active voice CaCaC to form *lamad*, 'he studied'.
* Intensive CiCeC to form *limed*, 'he taught'.
* Intensive passive CuCaC to form *lumad*, 'he was taught'.

### Agreement

We've seen that the subject noun and main verb in English must **agree** in number, meaning both are either singular or plural. Many languages also mark for **gender**, such as Spanish (*el chico*, *la chica*). The two genders in some languages are called masculine and feminine, and if languages have three genders, the third is called neuter. When some languages, such as the Bantu languages of Africa, have as many as 20 genders, we refer to them as **noun classes** instead of genders.

## 3.2 Finite-State Morphological Parsing (p. 59)



So we now have a goal: parse inputs as seen below.

Input | Parsed Output
--- | ---
cat | cat +N +Sg
cats | cat +N +Pl
cut | cut +V
cut | cut +N +Sg

The second column contains the word stem and its assorted morphological **features**, which specify additional information about the stem (e.g. ```+N``` means the word is a noun, ```+Sg``` means singular, ```+Pl``` is plural). Spanish might add a feature ```+Masc``` to indicate masculine words. We also already see ambiguity here, and for now we'll solve this by outputting all possible parses. See Chapter 5 for the task of disambiguating these options.

To build a parser, we'll at least need:
1. **Lexicon** - The list of stems and affixes, along with basic word class information about them.
2. **Morphotactics** - The model of which classes of morphemes can follow which other classes inside a word, e.g. the plural suffix follows its stem.
3. **Orthographic rules** - The spelling rules used to model the changes that occur in a word when we combine morphemes (e.g. *y* --> *ie* when we add plural -*s* to *city*).

The next section will discuss a simple lexicon representation, including using FSAs to model morphotactic knowledge. We'll then introduce the **finite-state transducer (FST)** to model morphological features in the lexicon, then use FSTs to model orthographical rules too.

## 3.3 Building a Finite-State Lexicon (p. 60)

"A lexicon is a repository for words." Listing every possible word is infeasible, so typically computational lexicons are structured as a list of stems and affixes as well as the morphotactics detailing how they can fit together. In designing a FSA for nominal inflection in English, we could use transitions on ```reg-noun```, ```plural -s```, ```irreg-pl-noun```, and ```irreg-sg-noun```. Verbs could be similarly split into multiple regular/irregular stem classes, plus suffixes -*ed* past, -*ed* participle, -*ing* participle, and 3sg -*s*.

English derivational morphology is even more complex. Consider the relatively simple case of English adjectives:
* big, bigger, biggest
* happy, happier, happiest, happily
* unhappy, unhappier, unhappiest, unhappily
* clear, clearer, clearest, clearly, unclear, unclearly
* cool, cooler, coolest, coolly
* red, redder, reddest
* real, unreal, really

(From Antworth (1990))

The naive hypothesis of optional prefix (*un*-), obligatory root, and optional suffix (-*er*, -*est*, or -*ly*) quickly fails when we recognize monstrosities like unbiggly, for example. So we'd need to construct classes with specific valid affixes, so ```adj-root```$_1$ includes adjectives that can have *un*- and -*ly*, while ```adj-root```$_2$ includes adjectives that can't. This complexity quickly creates large FSAs if we venture to model derivation for even a fragment of the possible English words.

These FSAs can now solve the problem of **morphological recognition**: determining whether or not an input string makes up a legitimate English word. We do this by taking the morphotactic FSAs and plugging in each possible sub-lexicon, expanding the arcs with all letters that make up each stem class.

## 3.4 Finite-State Transducers (p. 63)

A **finite-state transducer (FST)** is a finite automaton which maps between two sets of symbols. One could visualize this as a two-tape automaton which recognizes/generates pairs of strings. We can label each arc with two symbol strings, one for each tape. As an example, these two could be an input and output string. This means that the FST has a more general function than an FSA: the FSA defines a formal language as a set of strings, whereas an *FST defines a relation between sets of strings*.

You could think about the FST's complexity in a few different ways:
* Recognizer: The FST takes in a pair of strings and accepts only if the string-pair is in the string-pair language.
* Generator: The FST outputs pairs of strings of the language. The output is a yes or no, and a pair of output strings.
* Translator: The FST reads a string and outputs another string.
* Relater: The FST computes relations between sets.

All four of these have uses for this textbook. For morphological parsing, we'll apply the FST as a translator from strings of letters to strings of morphemes.

**Definition**: A finite-state transducer $F$ is defined as the 7-tuple $F=\langle Q,\Sigma,\Delta,q_0,F,\delta,\sigma\rangle$, where:
* $Q$ is a finite set of $N$ states $q_0,q_1,...,q_{N-1}$
* $\Sigma$ is a finite set corresponding to the input alphabet
* $\Delta$ is a finite set corresponding to the output alphabet
* $q_0 \in Q$ is the start state
* $F\subseteq Q$ is the set of final states
* $\delta(q,w)$ is the transition function between states. Given state $q \in Q$ and string $w \in \Sigma ^*$, $\delta(q,w)$ returns a set of new states $Q' \subseteq Q$. Thus $\delta:Q\times\Sigma ^* \to 2^Q$
* $\sigma(q,w)$ is the output function giving the set of possible output strings for each state and input. Given state $q \in Q$ and string $w \in \Sigma ^*$, $\sigma(q,w)$ returns a set of output strings $o \in \Delta ^*$. Thus $\sigma:Q\times\Sigma ^* \to 2^{\Delta ^*}$

FSTs are equivalent to **regular relations**, which are sets of pairs of strings, an obvious extension to regular languages (sets of strings). FSTs are closed under union but not under difference, compementation, nor intersection. FSTs also have two more quite useful properties:
* **Inversion**: The inversion of transducer $T$ ($T^{-1}$) switches the input and output labels. If $T$ maps from input alphabet $I$ to output alphabet $O$, $T^{-1}$ maps $O\to I$.
* **Composition**: If transducer $T_1$ maps $I_1 \to O_1$ and transducer $T_2$ maps $O_1 \to O_2$, then $T_1 \circ T_2$ maps $I_1 \to O_2$.

Inversion makes it easy to convert from FST-as-parser to FST-as-generator. Composition allows us to take two transducers in series and replace them with a single transducer. Intuitively, $T_1 \circ T_2(S)=T_2(T_1(S))$.

The **projection** of an FST is the FSA produced when we extract only one side of the relation. The projection of the left/upper side is the **upper** or **first** projection and the lower or right side gives the **lower** or **second** projection.

### Sequential Transducers and Determinism

FSTs as we've described them can be nondeterministic. This general case is quite slow, but unfortunately not all FSTs can be determinized, or $L(\text{D-FST})\not = L(\text{ND-FST})$. But **sequential transducers** are a subtype of FSTs that are deterministic on their input. Any state of a sequential transducer has at most one transition on each symbol in input alphabet $\Sigma$. Thus these can have $\epsilon$ only in the output symbols.

Because sequential transducers aren't necessarily sequential on their output, their inverse might not be sequential. Thus we need to specify the direction of transduction. Also note that the definition changes a bit so that we have $\delta:Q\times\Sigma ^* \to Q$ and $\sigma:Q\times\Sigma ^* \to \Delta ^*$.

A generalization that can then be made on sequential transducers is the **subsequential transducer**, which concatenates an additional output string when it reaches its final states. The importance of these two transducers (SQT, SSQT) comes from their determinism: processing them is linear to the input length. SSQTs also have algorithms for determinization and minimization, just as we had for FSA (see Mohri, 1997/2000). But what about models capable of handling ambiguity?

If we then extend SSQTs to deal with ambiguity, we arrive at the **$p$-subsequential** transducer. These allow for $p \geq 1$ final output strings to be associated with each final state, and thus handle a finite amount of ambiguity. The work of Mohri (1996, 1997) shows a number of tasks whose ambiguity can be limited in this way. These problems are **p-subsequentializable**, and thus can be determined and minimized. Luckily for us, this class of transducers includes many (not all) morphological rules.

## 3.5 FSTs For Morphological Parsing (p. 67)

Let's now actually work on morphological parsing. Given input *cats*, return *cat +N +Pl*, or given Spanish *bebo*, return *beber +V +PInd +1P +Sg* (present indicative first person singular verb). The **finite-state morphology** paradigm we'll use will represent a word as a correspondence between a **lexical level** (concatenation of morphemes making up a word) and **surface level** (concatenation of letters that spell the word).

To be continued...   
*--Last revised 6/21/2019--*